In [11]:
import numpy as np
import PIL.Image as Image
import os
import matplotlib.pyplot as plt
import matplotlib as mpl
from readPatchOutput import *
np.set_printoptions(threshold=np.inf)

repeat = 5
n_pic = 1
nV1 = 1024
nblock = 5
nE = 768
pixel = '16'
Epick = []
Ipick = []
for i in range(nblock):
    for j in range(nV1):
        if j <nE:
            Epick.append(i*nV1+j)
        else:
            Ipick.append(i*nV1+j)

In [12]:
res_fdr = '/root/autodl-tmp/ljy/resource_repeat/'
setup_fdr = '/root/autodl-tmp/ljy/resource_repeat/'
data_fdr = '/root/autodl-tmp/ljy/data_repeat_onlyLGN_nonoise/'
output_suffix = 'repeat_'

cfg = []
fr = []
F0_input = []
F1_input = []
for k in range(1,n_pic+1):
    #读取参数文件
    for i in range(1,repeat+1):
        with open(res_fdr+f'drifting_color_{k}_cfg.bin') as f:
            sf = np.fromfile(f,'f4',1)[0]
            ori = np.fromfile(f,'f4',1)[0]
            phase = np.fromfile(f,'f4',1)[0]
            c = np.fromfile(f,'f4',1)[0]
        cfg.append([sf,ori,phase,c])

        #读取文件
        with open(data_fdr+'sample_OutAndInputCount_'+output_suffix+str(i)+'_'+str(k)+'.bin') as f:
            sampleSize = np.fromfile(f,'u4',1)[0]
            sample_t0, sample_t1 = np.fromfile(f, 'f4', 2)
            sampleID = np.fromfile(f,'u4',sampleSize)
            sample_spikeCount = np.fromfile(f,'u4',sampleSize)
            F0 = np.fromfile(f,'f4',sampleSize)
            F1 = np.fromfile(f,'f4',sampleSize)    
        fr.append(sample_spikeCount)
        F0_input.append(F0)
        F1_input.append(F1)
    
fr = np.array(fr).reshape(n_pic, repeat, -1)
# fr = np.array(fr)
F0_input = np.array(F0_input).reshape(n_pic, repeat, -1)
F1_input = np.array(F1_input).reshape(n_pic, repeat, -1)
cfg = np.array(cfg)
nOri = 6

In [35]:
F0_input[1][:,:5]

array([[0.05149738, 0.        , 0.0499    , 0.        , 0.0499    ],
       [0.05147988, 0.        , 0.0499    , 0.        , 0.0499    ],
       [0.05148803, 0.        , 0.0499    , 0.        , 0.0499    ],
       [0.05150304, 0.        , 0.0499    , 0.        , 0.0499    ],
       [0.0514705 , 0.        , 0.0498    , 0.        , 0.0498    ],
       [0.05150266, 0.        , 0.0499    , 0.        , 0.0499    ],
       [0.05149738, 0.        , 0.0499    , 0.        , 0.0499    ],
       [0.0515043 , 0.        , 0.0499    , 0.        , 0.0499    ],
       [0.05148803, 0.        , 0.0499    , 0.        , 0.0499    ],
       [0.05148803, 0.        , 0.0499    , 0.        , 0.0499    ],
       [0.05148803, 0.        , 0.0499    , 0.        , 0.0499    ],
       [0.0514705 , 0.        , 0.0498    , 0.        , 0.0498    ],
       [0.05151892, 0.        , 0.0499    , 0.        , 0.0499    ],
       [0.05150266, 0.        , 0.0499    , 0.        , 0.0499    ],
       [0.05151239, 0.        , 0.

In [36]:
fr[1][:,:5]

array([[ 4,  4, 29,  1, 26],
       [ 3,  4, 28,  1, 26],
       [ 4,  4, 28,  1, 26],
       [ 4,  4, 29,  1, 26],
       [ 4,  4, 28,  1, 26],
       [ 3,  4, 29,  1, 26],
       [ 4,  4, 29,  1, 26],
       [ 4,  4, 29,  1, 26],
       [ 4,  4, 29,  1, 26],
       [ 4,  4, 29,  1, 26],
       [ 4,  4, 28,  1, 26],
       [ 4,  4, 28,  1, 26],
       [ 4,  4, 29,  1, 26],
       [ 4,  4, 29,  1, 26],
       [ 3,  4, 28,  1, 26],
       [ 4,  4, 28,  1, 26],
       [ 3,  4, 28,  1, 26]], dtype=uint32)

In [13]:
res_fdr = '/root/autodl-tmp/ljy/resource_repeat/'
setup_fdr = '/root/autodl-tmp/ljy/resource_repeat/'
data_fdr = '/root/autodl-tmp/ljy/data_repeat_onlyLGN_nonoise/'
output_suffix = 'repeat_'

nLGN = 512
nt = 40000

cfg = []
fr = []
LGN = np.empty((n_pic,repeat,nLGN,nt))
for k in range(1,n_pic+1):
    #读取参数文件
    for i in range(1,repeat+1):
        with open(res_fdr+f'drifting_color_{k}_cfg.bin') as f:
            sf = np.fromfile(f,'f4',1)[0]
            ori = np.fromfile(f,'f4',1)[0]
            phase = np.fromfile(f,'f4',1)[0]
            c = np.fromfile(f,'f4',1)[0]
        cfg.append([sf,ori,phase,c])

        #读取文件
        with open(data_fdr+'sample_spikeCount_'+output_suffix+str(i)+'_'+str(k)+'.bin') as f:
            sampleSize = np.fromfile(f, 'u4', 1)[0] #一共5120
            sample_t0, sample_t1 = np.fromfile(f, 'f4', 2)#t0是开始时间，t1是结束时间，我一共跑了1s
            nt = np.fromfile(f, 'u4', 1)[0]
            nLGN = np.fromfile(f, 'u4', 1)[0]
            LGN_spike_time = np.fromfile(f, 'u4', nLGN*nt)
            sampleID = np.fromfile(f, 'u4', sampleSize)#id排序是顺序的
            sample_spikeCount = np.fromfile(f, 'u4', sampleSize)
        LGN_spike_time = LGN_spike_time.reshape((nt,nLGN)).T
        LGN[k-1,i-1,:,:] = LGN_spike_time
        fr.append(sample_spikeCount)
    print(k)
fr = np.array(fr).reshape(n_pic, repeat, -1)
# LGN = np.aray(LGN).reshape(n_pic,repeat,nLGN,nt)
cfg = np.array(cfg)
nOri = 6

1


AttributeError: module 'numpy' has no attribute 'aray'

In [5]:
nLGN = 512
nt = 40000
LGN = np.empty((n_pic,repeat,nLGN,nt))

In [17]:
LGN_sum = LGN.sum(axis=3)

In [22]:
LGN_sum[0,:,200:211]

array([[492., 512., 518., 510., 555., 523., 522., 488., 524., 536., 485.],
       [492., 512., 518., 509., 555., 522., 522., 488., 524., 535., 486.],
       [492., 512., 518., 510., 555., 523., 523., 488., 524., 536., 487.],
       [492., 512., 518., 510., 555., 522., 522., 487., 524., 537., 486.],
       [492., 512., 518., 510., 555., 522., 523., 488., 524., 536., 487.]])

In [31]:
np.arange(nt)[LGN[0,1,0,:]==1][:30],np.arange(nt)[LGN[0,2,0,:]==1][:30]

(array([ 303,  305,  310,  312,  341, 1060, 1064, 1074, 1146, 1235, 1245,
        1253, 1271, 1273, 1285, 1310, 1320, 1332, 1348, 1360, 1398, 2059,
        2079, 2114, 2120, 2141, 2146, 2148, 2156, 2219]),
 array([ 302,  305,  309,  312,  341, 1060, 1064, 1073, 1145, 1235, 1245,
        1253, 1271, 1273, 1284, 1310, 1320, 1332, 1348, 1360, 1397, 2058,
        2079, 2113, 2119, 2141, 2146, 2147, 2156, 2219]))

In [33]:
abs(LGN[0,1,:,:].sum(axis=1) - LGN[0,2,:,:].sum(axis=1)).sum()

108.0

In [25]:
abs(LGN[0,1,:,:] - LGN[0,3,:,:]).sum(axis=1)

array([ 796.,  804.,  652.,  732.,  354.,  906.,  903.,  894.,  935.,
        864.,  886.,  954.,  816.,  760.,  546.,  924.,  882.,  304.,
        981.,  894.,  278.,  748.,  856.,  352.,  822.,  876.,  928.,
        850.,  654.,  880.,  916.,  834.,  822.,  836.,  881.,  832.,
        504.,  830.,  947.,  807.,  928.,  830.,  636.,  406.,  960.,
        866.,  868.,  806.,  822.,  931.,  878.,  845.,  949.,  866.,
        884.,  840.,  865.,  776.,  857.,  406.,  829.,  938.,  620.,
        930.,  638.,  938.,  869.,  544.,  841.,  859.,  792.,  860.,
        912.,  864.,  836.,  874.,  764.,  846.,  452.,  847.,  274.,
        856.,  204.,  902.,  750.,  894.,  903.,  804.,  740.,  910.,
        926.,  956.,   78.,  812.,  192.,  970.,  858.,  925.,  941.,
        476.,  906.,  458.,  784.,  852.,  246.,  822.,  879.,  880.,
        690.,  780.,  828.,  884.,  786.,  952.,  883.,  328.,  746.,
        888.,  819.,  926.,  912.,  938.,  904.,  893.,  997.,  967.,
        972.,  929.,